In [ ]:
!pip install flask_jwt_extended
!pip install flask


# Flask API - EHR

In [ ]:
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, jwt_required, create_access_token, get_jwt_identity
 

app = Flask(__name__)
app.config['JWT_SECRET_KEY'] = 'tmObbSxOOwIwh12IbYIM8Ouojt14FrZEnQ4JEJbluVxastT6HD3YKMldFvRUv6P'
jwt = JWTManager(app)
 
# The route() function of the Flask class is a decorator, 
# which tells the application which URL should call 
# the associated function.





@app.route('/login', methods=['POST'])
def login():
    if request.is_json:
        user = request.json['user']
        password = request.json['password']
    else:
        user = request.form['user']
        password = request.form['password']

    
    if check_login(user, password):
        access_token = create_access_token(identity=user)
        return jsonify(message='Login Successful', access_token=access_token)
    else:
        return jsonify('Bad email or Password'), 401



@app.route('/', methods=['GET'])
@jwt_required()
def hello_world():
    user_id = get_jwt_identity()
    print(user_id)

    return "Welcome to EHR database, you're logged in as: " + user_id



#lab data

@app.route('/lab_data', methods=['GET'])
@jwt_required()
def lab_data():
    
    user_id = get_jwt_identity()

    data = get_lab_data(11, lab_df)
    data_json = data.to_json(orient='records')

    return jsonify(data_json)



#patient basic information





app.run(host='0.0.0.0', port=8080)

## Patient data

In [ ]:
#patient basic information - not yet ready

@app.route('/patient_data', methods=['GET'])
@jwt_required()
def patient():
    user_id = get_jwt_identity()
    

    return x

## App run

In [ ]:
app.run(host='0.0.0.0', port=8080)

# Read csv files

In [ ]:
import pandas as pd

lab_df =  pd.read_csv("lab_results_10_tests_patients_1_15_NA.csv")
lab_df['date'] = pd.to_datetime(lab_df['date'])


patient_df =  pd.read_csv("patient_information.csv")



## Functions

In [ ]:
def check_login(input_user, input_pass, df):
    if input_user in patient_df.username.values:

        password = df[df.username==input_user]['password'].item()
        
        if input_pass == password:
            print('Login successful')
            return True
        else:
            return False

    else:
        print('Username not available')
        return False


Lab data

In [ ]:
def get_lab_data(n, df):
    patient11 = df[df.patient_id == n]
    patient11 = patient11.copy()
    patient11.sort_values(by='date', inplace=True)
    patient11.reset_index(inplace=True, drop=True)

    return patient11